# Linear Regression

A linear model makes a prediction by computing a weighted sum of the input features. Given N observations from the training set, each observation has M features and each feature would be assigned a weight. 

Training a Linear Regression means finding the weights that minimize some measure of error. The most common performance measure of a regression model is the Mean Square Error (MSE).

$\begin{bmatrix}\hat{y_{1}}\\\hat{y_{2}}\\...\\\hat{y_{N}}\end{bmatrix}$ = $\begin{bmatrix}x_{1,1}   x_{1,2} ... x_{1,M}\\x_{2,1}   x_{2,2} ... x_{2,M}\\...\\x_{N,1} x_{N,2} ... x_{N,M}\end{bmatrix}$ 
$\begin{bmatrix}w_{1}\\w_{2}\\...\\w_{M}\end{bmatrix}$

$Y = X.W$

MSE = $\dfrac{1}{N} \sum^{N} {(y_{n}-\hat{y_{n}})}^2$

We minimize the MSE by differentiating it against each of the weights, and setting the derivative equal to zero.<br>
MSE = $\dfrac{1}{N} \sum^{N} {(y_{n}-\sum^{M} w_{m} x_{nm}   )}^2$ <br>

$\dfrac{d(MSE)}{dw_{k}}$ = $\dfrac{1}{N} \sum^{N} \dfrac{d{(y_{n}-\sum^{M} w_{m} x_{nm})}^2}{dw_{k}}$

Using chain rule $\dfrac{dz}{dx} = \dfrac{dz}{dy} . \dfrac{dy}{dx}$, and setting the derivative to 0, we have ...<br>
$\dfrac{d(MSE)}{dw_{k}}$ = $\sum^{N} \dfrac{d{(y_{n}-\sum^{M} w_{m} x_{nm})}^2}{d(\sum^{M} w_{m} x_{nm})} . \dfrac{d(\sum^{M} w_{m} x_{nm})}{dw_{k}} = 0$ <br><br>

$\sum^{N} -2{(y_{n}-\sum^{M} w_{m} x_{nm}   )}.x_{nk}$ = 0 <br>
$\sum^{N} y_{n}.x_{nk} =\sum^{N}\sum^{M} w_{m} x_{nm}.x_{nk}$<br><br>
$\sum^{N}_{n=1}\sum^{M}_{k=1} y_{n}.x_{nk} =\sum^{N}_{n=1}\sum^{M}_{k=1}\sum^{M}_{m=1}  w_{m} x_{nm}.x_{nk}$<br><br>

In Matrix form:<br>
$Y^{T}X = W^{T}{(X^{T}.X)}$<br>
$W^{T} = {(X^{T}.X)}^{-1}(Y^{T}X)$<br>
$W = {(X^{T}.X)}^{-1}(X^{T}Y)$

The above is called the **NORMAL EQUATION**.

In [1]:
### Example 
import pandas as pd
import matplotlib.pyplot as plt
import sys,os
import numpy as np
sys.path.append(os.path.realpath('..'))

data = pd.read_csv("./machine_learning_examples/linear_regression_class/data_2d.csv",header=None)
X = data.iloc[:,0:2].values
Y = data.iloc[:,2].values
Y = np.reshape(Y, (Y.shape[0],-1))

bias = np.ones((X.shape[0],1))
X = np.hstack((bias,X))
W = (np.linalg.inv((X.T).dot(X))).dot((X.T).dot(Y))
Y_pred = X.dot(W)
print(W)

MSE = ((Y.T).dot(Y) + (Y_pred.T).dot(Y_pred) - 2*(Y.T).dot(Y_pred))/X.shape[0]
print(MSE)
from sklearn.metrics import r2_score
r2_score(Y,Y_pred)

[[1.46191241]
 [2.01666793]
 [2.96985048]]
[[22.35949035]]


0.9980040612475777

In [2]:
from sklearn.linear_model import LinearRegression

lin_reg = LinearRegression()
lin_reg.fit(X[:,1:],Y)
lin_reg.intercept_,lin_reg.coef_

(array([1.46191241]), array([[2.01666793, 2.96985048]]))

By default, sklearn LinearRegressor automatically adds the bias terms to the matrix.

# Gradient Descent

Gradient Descent is a better approach for Linear Regression models with a large number of features and many training instances. 

Gradient descent firstly picks a random point of initialization and moves in the direction of descending gradient iteratively until the minimum is reached. The size of the steps is called the learning rate hyperparameter and should be large enough to ensure quick convergence but not too large as to jump across and miss the minimum. 

For functions where there are many alleys, depending on the initialization point, the GD may converge to a local minimum instead of a global minimum, which is not ideal. Fortunately, the MSE for a LR is a convex function and convergence to a global minimum is guaranteed.

![alt](images/gradient_descent.jpg)

Obtain the partial derivatives for the MSE with respect to each weight...

$\dfrac{d(MSE)}{dw_{k}} = \dfrac{2}{N} \sum^{N} {(\sum^{M} w_{m} x_{nm}-y_{n})}.x_{nk}$<br>

$\dfrac{d(MSE)}{dw} = \sum^{M}_{k=1} \dfrac{d(MSE)}{dw_{k}} = \dfrac{2}{N} [\sum^{N}_{n=1}\sum^{M}_{k=1}\sum^{M}_{m=1}  w_{m} x_{nm} x_{nk}-\sum^{N}_{n=1}\sum^{M}_{k=1} y_{n}.x_{nk}]$<br>

In Matrix form:<br>
$\dfrac{2}{N} [W^{T}(X^{T}.X) - Y^{T}.X] = \dfrac{2}{N} [(W^{T}X^{T}).X - Y^{T}.X]$ <br>
$\dfrac{2}{N} [(W^{T}X^{T} - Y^{T}).X]= \dfrac{2}{N} [(X.W - Y)^{T}.X]$<br><br>
$\dfrac{d(MSE)}{dw} = \dfrac{2}{N} [X^{T}(X.W - Y)]$ ... is the gradient vector<br>

If the gradient vector is positive, we go in the opposite direction to go downhill...

$w^{next} = w - \lambda \dfrac{d(MSE)}{dw}$, where $\lambda$ is the learning rate.

In [3]:
# Initialize random weights
W = np.random.rand(X.shape[1],1)
gradient_vector = (X.T.dot(X.dot(W)-Y))*2/Y.shape[0]
print("Starting point:",W)
print("Gradient=",sum(gradient_vector))
print("\n")



lr = 0.00015
iterations = 0
# Batch gradient Descent, training on the whole dataset
while iterations<200000:
    W = W - lr*gradient_vector
    gradient_vector = (X.T.dot(X.dot(W)-Y))*2/Y.shape[0]
    iterations+=1
    
print("W=",W)
print("Gradient=",sum(gradient_vector))

Starting point: [[0.93433524]
 [0.32665939]
 [0.73010235]]
Gradient= [-50486.81091261]


W= [[1.46177305]
 [2.01666901]
 [2.9698517 ]]
Gradient= [-3.73488206e-05]


The results obtained with GD gets very close to that of the Normal Equation. Given enough time, it will reach the optimal. But from this point on it will take even longer, because the rate of convergence decreases as we get closer to the minimum. 

# Stochastic Gradient Descent

When the training set (N) is large and there are many features (M) the gradient descent algorithm could become slow because at every iteration ... computation is done on vectors of size Mx1 and matrices of size MxN. 

Stochastic Gradient Descent is faster because at each iteration it picks a random instance from the training set and computes the gradients based on that one instance. 

Another variation is the Mini-batch Gradient Descent, which obtain the gradients based on a small random sample of instances. The main advantage of Mini-batch GD over SGD is that it gets a performance boost from hardware optimized matrix operations, especially when using GPUs.


In [4]:
# Mini-batch SGD
lr = 0.00015
iterations = 0
W = np.random.rand(X.shape[1],1)

# We can add a threshold value of 0.05, to stop early once the gradient gets close to 0
while iterations<200000:
    r = np.random.randint(Y.shape[0])
    X_r = X[r:r+5]
    Y_r = Y[r:r+5]
    X_r = np.reshape(X_r, (X_r.shape[0],-1))
    
    gradient_vector = (X_r.T.dot(X_r.dot(W)-Y_r))*2/Y_r.shape[0]
    if (0 < sum(gradient_vector)) and (sum(gradient_vector) < 0.05):
        break 
    W = W - lr*gradient_vector
    iterations+=1
    
print("W=",W)
print("Gradient=",sum(gradient_vector))

W= [[0.39456283]
 [1.96388006]
 [3.02439826]]
Gradient= [0.04842737]


# Regularized Linear Regression

To avoid overfitting in linear regression, we need to regularize the weights of the model. This means adding a term to the cost function that penalizes large weights.<br>

The hyperparameter $\alpha$ controls how much to regularize the model. The bias term $w_{1}$ is not regularized. <br>


#### Ridge Regression (L1)
Cost = $\dfrac{1}{N} \sum^{N} {(y_{n}-\hat{y_{n}})}^2 + \dfrac{\alpha}{2} \sum^{N}_{i=2} w^{2}_{i}$ <br>
Ridge regression keeps all features but shrinks their weights to helps to reduce the model complexity and multi-collinearity.<br> 
 

#### Lasso Regression (L2)
Cost = $\dfrac{1}{N} \sum^{N} {(y_{n}-\hat{y_{n}})}^2 + \alpha \sum^{N}_{i=2} |w_{i}|$ <br>
Lasso regression not only helps in reducing over-fitting but it can help in feature selection and it outputs a sparse model, unimportant predictors are left out of the model (i.e weight = 0). <br> 

#### ElasticNet 
Cost = $\dfrac{1}{N} \sum^{N} {(y_{n}-\hat{y_{n}})}^2 + \dfrac{\alpha(1-r)}{2} \sum^{N}_{i=2} w^{2}_{i} + r\alpha \sum^{N}_{i=2} |w_{i}|$ <br>


It is almost always preferable to have some regularization, so generally you should avoid plain Linear Regression. Ridge is a good default, but if you suspect only a few features are useful then use Lasso or ElasticNet. ElasticNet is better than Lasso when several features are strongly correlated. Similar to plain Linear Regression, Gradient Descent can be used to find the optimal weights. 

In [5]:
# Stochastic Gradient Descent regressor, with L1 (Ridge) regularization in sklearn
from sklearn.linear_model import SGDRegressor

sdg_reg = SGDRegressor(max_iter=200000,eta0=0.00015,penalty='l1')
sdg_reg.fit(X[:,1:],Y.ravel())
Y_pred = sdg_reg.predict(X[:,1:])

from sklearn.metrics import r2_score
sdg_reg.intercept_,sdg_reg.coef_,r2_score(Y,Y_pred)

(array([0.04021992]), array([2.02240985, 2.9879615 ]), 0.9979741296257675)


# Logistic Regression

Logistic Regression is a binary classification algorithm. It estimates the probability that an instance belongs to a particular class. The probability is rounded to 0 or 1 to obtain the final answer.

Logistic Regression model computes the sigmoid of the weighted sum of input features $\sigma (X_{NxM}.W_{Mx1})$. 

![alt](images/logistic_function.jpg)

The objective of training is to set parameter vector W so that the model estimates high probabilities for positive instances (y = 1) and low probabilities for negative instances (y = 0).<br>Large deviations between $y_{n}$ and $\hat{y_{n}}$ should be penalized.

CrossEntropy = ... <br>
J = $-\dfrac{1}{N} \sum^{N} [y_{n}log(\sigma(\hat{y_{n}})) + (1-y_{n})log(1-\sigma(\hat{y_{n}}))]$


- If $y_{n} = 1$, the log value approaches $-\infty$ as $\hat{y_{n}}$ approaches zero.  
- If $y_{n} = 0$, the log value approaches $-\infty$ as $\hat{y_{n}}$ approaches 1. 

To minimize the cost function, we obtain partial derivatives w.r.t each of the weights.

$\dfrac{dJ}{dw_{k}}$ = $-\dfrac{1}{N} \sum^{N} \dfrac{d(y_{n}log(\sigma(\hat{y_{n}})) + (1-y_{n})log(1-\sigma(\hat{y_{n}}))])}{d(\sigma(\hat{y_{n})})} \dfrac{d(\sigma(\hat{y_{n}}))}{dw_{k}}$

$\dfrac{dJ}{dw_{k}}$ = $-\dfrac{1}{N} \sum^{N} \dfrac{y_{n}(1+e^{-\hat{y_{n}}})^{2} - (1+e^{-\hat{y_{n}}})}{e^{-\hat{y_{n}}}} \dfrac{d(\sigma(\hat{y_{n}}))}{dw_{k}}$

$\dfrac{dJ}{dw_{k}}$ = $-\dfrac{1}{N} \sum^{N} \dfrac{y_{n}(1+e^{-\hat{y_{n}}})^{2} - (1+e^{-\hat{y_{n}}})}{e^{-\hat{y_{n}}}} \dfrac{d(\sigma(\hat{y_{n}}))}{d\hat{y_{n}}} \dfrac{{d\hat{y_{n}}}}{dw_{k}}$

$\dfrac{dJ}{dw_{k}}$ = $-\dfrac{1}{N} \sum^{N} \dfrac{y_{n}(1+e^{-\hat{y_{n}}})^{2} - (1+e^{-\hat{y_{n}}})}{e^{-\hat{y_{n}}}} \dfrac{e^{-\hat{y_{n}}}}{(1+e^{-\hat{y_{n}}})^{2}} x_{nk}$

$\dfrac{dJ}{dw_{k}}$ = $\dfrac{-1}{N} \sum^{N} \left[y_{n} - \dfrac{1}{(1+e^{-\hat{y_{n}}})}\right] x_{nk}$

$\dfrac{dJ}{dw} = \sum^{M}_{k=1} \dfrac{dJ}{dw_{k}} = \dfrac{-1}{N} \sum^{M}_{k=1} \sum^{N}_{n=1} \left[y_{n} - \dfrac{1}{(1+e^{-\hat{y_{n}}})}\right] x_{nk}$

In matrix form:
$\dfrac{dJ}{dw} = \dfrac{1}{N} X^{T}[\sigma(XW)-Y]$ ... is the gradient vector.<br><br>

Similar to Linear Regression, the optimal weights to a Logistic Regression model can be found using Gradient Descent and the model can be regularized using L1 or L2 penalities.

Next we implement a Logistic Regression model on the iris dataset, trying to detect the *Iris virginica*.

In [6]:
from sklearn import datasets
iris = datasets.load_iris()
print(iris.feature_names)
X = iris.data
Y = (iris.target==2).astype(int) # y==2 is iris virginica
Y = np.reshape(Y, (Y.shape[0],-1))

# Add bias terms and initialize random weights
bias = np.ones((X.shape[0],1))
X = np.hstack((bias,X.copy()))
W = np.random.rand(X.shape[1],1)
def sigmoid(X,W):
    return 1/(1+np.exp(-(X.dot(W))))

gradient_vector = X.T.dot(sigmoid(X,W)-Y)/Y.shape[0]
print("Gradient=",gradient_vector)
print("\n")

lr = 0.1
iterations = 0
while iterations<100:
    W -= lr*gradient_vector
    gradient_vector = X.T.dot(sigmoid(X,W)-Y)/Y.shape[0]
    iterations+=1
    
    
print("W=",W)
print("Gradient=",sum(gradient_vector))
Y_pred = np.round(sigmoid(X,W))
from sklearn.metrics import accuracy_score
accuracy_score(Y, Y_pred)

['sepal length (cm)', 'sepal width (cm)', 'petal length (cm)', 'petal width (cm)']
Gradient= [[0.6596706 ]
 [3.61170314]
 [2.04354834]
 [1.8933321 ]
 [0.52079459]]


W= [[-0.10393231]
 [-0.74857426]
 [-0.67264665]
 [ 0.93949494]
 [ 1.32211633]]
Gradient= [-0.01098803]


0.9733333333333334

## Softmax Regression

Logistic Regression can be generalized to support multiple classes directly. Each class will have its own parameter vector $W_{m}$. The paramter matrix will be of dimension MxK (K classes, M parameters for each).

Softmax Function: <br>
$s(\hat{y}_{nk}) = \dfrac{e^{\hat{y}_{nk}}}{\sum^{K}_{j=1} e^{\hat{y}_{nj}} }$

$\hat{Y}_{NxK} = X_{NxM}W_{MxK}$ .......... where $y_{nk}$ is the probablity that instance n belongs to class k. <br><br>

Given instance n: <br>
$\hat{y}_{n} = \begin{pmatrix} \hat{y}_{n1} & \hat{y}_{n2} &  ...  & \hat{y}_{nK} \end{pmatrix} = 
\begin{pmatrix} x_{n1} & x_{n2} & ... & x_{nM} \end{pmatrix}
\begin{pmatrix} w_{11} & ... & w_{1K} \\ ... & ... & ... \\ w_{M1} & ... & w_{MK} \end{pmatrix}$  

$\hat{y}_{nj} = 
\begin{pmatrix} x_{n1} & x_{n2} & ... & x_{nM} \end{pmatrix}
\begin{pmatrix} w_{1j} \\ ... \\ w_{Mj} \end{pmatrix}$  

The final predicted class is the one with the highest probability out of all the classes. <br><br>

___

The optimal Weight matrix is obtained by minimizing the CrossEntropy ... <br>
J = $-\dfrac{1}{N} \sum^{N} \sum^{K}[y_{nk}log(s({\hat{y}_{nk}}))]$

$\dfrac{dJ}{dw_{mj}} = \dfrac{-1}{N} \sum^{N}_{n=1} \sum^{K}_{k=1} \dfrac{d(y_{nk}log(s({\hat{y}_{nk}})))}{ds(\hat{y}_{nk})} \dfrac{ds(\hat{y}_{nk})}{d(\hat{y}_{nj})} \dfrac{d(\hat{y}_{nj})}{dw_{mj}} $
- We can differentiate $s(\hat{y}_{nk})$ w.r.t any $\hat{y}_{nj}$ for $j \in K$. <br><br>
____


Partial Derivatives: <br>
$\dfrac{d(y_{nk}log(s({\hat{y}_{nk}})))}{ds(\hat{y}_{nk})} = \dfrac{y_{nk}}{s(\hat{y}_{nk})}$<br><br>

If $k=j...$ $\ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ $ If $k \neq j$...<br>
$\dfrac{ds(\hat{y}_{nk})}{d(\hat{y}_{nj})} = s(\hat{y}_{nk})[1-s(\hat{y}_{nk})]$ $\ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \  $          $\dfrac{ds(\hat{y}_{nk})}{d(\hat{y}_{nj})} = s(\hat{y}_{nk})[0-s(\hat{y}_{nj})]$ 

Let $\delta_{jk} = \begin{cases}
      1 & \text{if k=j}\\
      0 & \text{otherwise}
    \end{cases}$ $\ \ \ \ \ => \ \ \ $ $\dfrac{ds(\hat{y}_{nk})}{d(\hat{y}_{nj})} = s(\hat{y}_{nk})[\delta_{jk}-s(\hat{y}_{nj})]$ <br><br>
    
$\dfrac{d(\hat{y}_{nj})}{dw_{mj}} = x_{nm}$<br><br>

Thus, we have ...
$\dfrac{dJ}{dw_{mj}} = \dfrac{-1}{N} \sum^{N}_{n=1} \sum^{K}_{k=1} \dfrac{y_{nk}}{s(\hat{y}_{nk})} s(\hat{y}_{nk})[\delta_{jk}-s(\hat{y}_{nj})] x_{nm}$ <br><br>

$\sum^{K}_{k=1} y_{nk} [\delta_{jk}-s(\hat{y}_{nj})] = \sum^{K}_{k=1} y_{nk} \delta_{jk} - \sum^{K}_{k=1} y_{nk} s(\hat{y}_{nj}) = y_{nj} \delta_{jj} -  s(\hat{y}_{nj}) \sum^{K}_{k=1} y_{nk} = y_{nj} -  s(\hat{y}_{nj}) $
- Only one target class = 1, the rest should be 0.
- $\delta_{jk}$ = 1 when j=k.

$\dfrac{dJ}{dw_{mj}} = \dfrac{1}{N} \sum^{N}_{n=1} [s(\hat{y}_{nj}) - y_{nj}] x_{nm} $ <br><br>

In matrix form:<br>
$\dfrac{dJ}{dw} = \dfrac{1}{N} \sum^{K}_{j=1} \sum^{M}_{m=1} \sum^{N}_{n=1} [s(\hat{y}_{nj}) - y_{nj}] x_{nm} = \dfrac{1}{N} X^{T}[S-Y]$<br>



In [7]:
from sklearn import datasets
from sklearn.linear_model import LogisticRegression
iris = datasets.load_iris()
print(iris.feature_names)
X = iris.data
Y = iris.target

softmax_reg = LogisticRegression(multi_class="multinomial",solver='sag',penalty='none',max_iter=10000)
softmax_reg.fit(X,Y.ravel())
Y_pred = softmax_reg.predict(X)

from sklearn.metrics import accuracy_score
bias = softmax_reg.intercept_
bias = bias.reshape((bias.shape[0],-1))
W = np.hstack((bias,softmax_reg.coef_))

accuracy_score(Y, Y_pred)
print("Final W:\n",W)
accuracy_score(Y, Y_pred)

['sepal length (cm)', 'sepal width (cm)', 'petal length (cm)', 'petal width (cm)']
Final W:
 [[ 1.17885699  2.50512593  5.06738172 -7.09848929 -3.48547412]
 [ 8.39520276  0.40501534 -0.01029974  0.13094853 -4.06613722]
 [-9.57405975 -2.91014127 -5.05708197  6.96754076  7.55161134]]


0.98

In [8]:
from tensorflow import keras
X = iris.data
Y = iris.target

# Add bias terms, one-hot-encode, initialize random weights
bias = np.ones((X.shape[0],1))
X = np.hstack((bias,X.copy()))
Y = keras.utils.to_categorical(Y)
W = np.random.rand(X.shape[1],Y.shape[1])

lr = 0.1
iterations = 0
def softmax(Y):
    expY = np.exp(Y)
    return expY/expY.sum(axis=1,keepdims=True)

while iterations<1000:
    S = softmax(X.dot(W))
    gradient_vector = (X.T.dot(S-Y))/Y.shape[0]
    W -= lr*gradient_vector
    iterations+=1
    
print("W=",W)
print("Gradient=",gradient_vector)

Y_pred = X.dot(W)
Y_pred = np.floor(Y_pred/np.max(Y_pred,axis=1)[:,None])
Y_pred[Y_pred!=1]=0
accuracy_score(Y, Y_pred)

W= [[ 1.45230015  0.79545567 -0.8516103 ]
 [ 1.39075866  1.33232041 -0.98007906]
 [ 2.61750391  0.50997802 -1.16714139]
 [-2.32179923  0.27935419  3.26906507]
 [-1.03453791 -0.69890885  3.09895392]]
Gradient= [[-0.00136449 -0.00454682  0.00591131]
 [-0.00317449 -0.00203603  0.00521052]
 [-0.00672603 -0.00156329  0.00828933]
 [ 0.00881019  0.00162103 -0.01043121]
 [ 0.00423791  0.0052446  -0.00948251]]


0.9866666666666667